# STEP 1: LOAD DATA SET & CONSTRUCT ragas_evaluation_dataset.xlsx

In [15]:
import os
import pandas as pd
from pathlib import Path

# --- 1. CONFIGURATION ---
# Mount Drive
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

DATA_ROOT = Path("/content/drive/MyDrive/Project-AI-Eval/Test-Data/")
OUTPUT_FILE = "ragas_evaluation_dataset.xlsx"

# --- 2. PROCESSING FUNCTION ---
def load_case_data(root_path):
    data_entries = []

    # Get all subfolders (001, 002, etc.) and sort them
    if not root_path.exists():
        print(f"❌ Error: Path {root_path} does not exist!")
        return []
    case_folders = sorted([f for f in root_path.iterdir() if f.is_dir()])

    print(f"📂 Found {len(case_folders)} case folders. Processing...")

    for folder in case_folders:
        case_id = folder.name  # e.g., "001"

        # Paths
        query_path = folder / "query.txt"
        gt_path = folder / "ground_truth.txt"
        images_dir = folder / "images"

        # 1. Read Query (User Input)
        user_input = ""
        if query_path.exists():
            try:
                with open(query_path, "r", encoding="utf-8") as f:
                    user_input = f.read().strip()
            except Exception as e:
                print(f"⚠️ Error reading query for {case_id}: {e}")
        else:
            print(f"⚠️ Missing query.txt for case {case_id}")
            continue

        # 2. Read Ground Truth
        ground_truth = ""
        if gt_path.exists():
            try:
                with open(gt_path, "r", encoding="utf-8") as f:
                    ground_truth = f.read().strip()
            except Exception as e:
                print(f"⚠️ Error reading ground_truth for {case_id}: {e}")

        # 3. Process Images (The "Image Context" Step)
        image_notes = []
        if images_dir.exists() and images_dir.is_dir():
            # Get valid image files
            images = [img.name for img in images_dir.iterdir()
                      if img.suffix.lower() in ['.png', '.jpg', '.jpeg']]

            if images:
                # Sort images to maintain order (e.g. p1 before p2)
                images.sort()

                # Format: [User provided image: /path/to/image.png]
                for img_name in images:
                    full_img_path = str(images_dir / img_name)
                    image_notes.append(f"[User provided image: {full_img_path}]")

        # 4. Construct Final User Input for Ragas
        # Combine Text Query + Image Contexts
        final_user_input = user_input
        if image_notes:
            final_user_input += "\n\n" + "\n".join(image_notes)

        # 5. Append to List
        data_entries.append({
            "case_id": case_id,
            "user_input": final_user_input,
            "ground_truth": ground_truth
        })

    return data_entries

# --- 3. EXECUTION ---
print("🚀 Starting Data Preparation...")
dataset_list = load_case_data(DATA_ROOT / "extracted-data")

if dataset_list:
    df = pd.DataFrame(dataset_list)

    # Save to Excel
    df.to_excel(OUTPUT_FILE, index=False)

    print(f"\n✅ Success! Processed {len(df)} cases.")
    print(f"📁 Saved to: {OUTPUT_FILE}")
    print("\n--- Preview of Row 0 ---")
    print(df.iloc[0]['user_input'])
else:
    print("❌ No data found. Check your folder path.")

🚀 Starting Data Preparation...
📂 Found 33 case folders. Processing...

✅ Success! Processed 33 cases.
📁 Saved to: ragas_evaluation_dataset.xlsx

--- Preview of Row 0 ---
A 53-year-old woman presented with fever, cough, and malaise after returning from a visit to Lahore. On examination, her temperature was 38°C and she had a rash on her upper chest. A chest X-ray showed patchy basal consolidation and a full blood count revealed a relative lymphocytosis. Malaria films were negative. Blood cultures were drawn and later grew gram-negative bacilli.


# STEP 2: APPEND MODEL ANSWER to ragas_evaluation_dataset.xlsx

In [16]:
import re
import pandas as pd
from pathlib import Path

def append_gwen_results(
    source_file,
    result_file,
    model_tag: str,
    answer_col: str = "answer",
    case_id_col: str = "case_id",
    context_col: str | None = None,   # Pass in context if the model uses RAG
):
    # -----------------------------
    # 0. NORMALIZE INPUTS
    # -----------------------------
    source_file = Path(source_file)
    result_file = Path(result_file)

    if not source_file.exists():
        raise FileNotFoundError(f"❌ Source file not found: {source_file}")
    if not result_file.exists():
        raise FileNotFoundError(f"❌ Result file not found: {result_file}")

    if not re.match(r"^[a-zA-Z0-9_]+$", model_tag):
        raise ValueError(f"❌ Invalid model_tag: {model_tag}")

    disease_col = f"{model_tag}_answer_disease"
    reasoning_col = f"{model_tag}_answer_reasoning"
    context_out_col = f"{model_tag}_context" if context_col else None

    print(f"🔧 Processing model: {model_tag}")

    # -----------------------------
    # 1. LOAD SOURCE
    # -----------------------------
    try:
        df_src = pd.read_excel(source_file)
    except Exception:
        df_src = pd.read_csv(source_file)

    for col in [case_id_col, answer_col]:
        if col not in df_src.columns:
            raise KeyError(f"❌ Column '{col}' missing in {source_file}")

    if context_col and context_col not in df_src.columns:
        raise KeyError(f"❌ Context column '{context_col}' missing in {source_file}")

    # -----------------------------
    # 2. ROBUST EXTRACTION
    # -----------------------------
    # Paterns of the model's answer
    PATTERNS = [
        r"\*\*Predicted disease:\*\*\s*(.*?)\s*\*\*Reason:\*\*\s*(.*)",
        r"Predicted disease:\s*(.*?)\s*Reason:\s*(.*)",
        r"Predicted disease:\s*(.*?)\s*Reasoning:\s*(.*)",
        r"Disease:\s*(.*?)\s*Reason:\s*(.*)",
        r"Diagnosis:\s*(.*?)\s*Reason:\s*(.*)",
        r"predicted disease is\s*(.*?)\s*(?:because|as|due to)\s*(.*)",
    ]

    def extract(text):
        text = str(text).strip()
        for p in PATTERNS:
            m = re.search(p, text, re.IGNORECASE | re.DOTALL)
            if m:
                return m.group(1).strip(), m.group(2).strip()
        return "", ""

    extracted = df_src[answer_col].apply(lambda x: pd.Series(extract(x)))
    extracted.columns = [disease_col, reasoning_col]
    df_src = pd.concat([df_src, extracted], axis=1)

    # -----------------------------
    # 3. SANITY CHECK
    # -----------------------------
    empty_rate = (df_src[disease_col] == "").mean()
    if empty_rate > 0.3:
        raise ValueError(
            f"❌ Extraction failed for {empty_rate:.0%} of rows "
            f"(format drift or broken regex)"
        )

    df_src[case_id_col] = df_src[case_id_col].astype(str)

    # -----------------------------
    # 4. LOAD RESULT FILE
    # -----------------------------
    df_res = pd.read_excel(result_file)
    if case_id_col not in df_res.columns:
        raise KeyError(f"❌ '{case_id_col}' missing in result file")

    df_res[case_id_col] = df_res[case_id_col].astype(str)

    # -----------------------------
    # 5. OVERWRITE EXISTING COLUMNS
    # -----------------------------
    cols_to_drop = [disease_col, reasoning_col]
    if context_out_col:
        cols_to_drop.append(context_out_col)

    for col in cols_to_drop:
        if col in df_res.columns:
            print(f"⚠️ Overwriting existing column: {col}")
            df_res = df_res.drop(columns=[col])

    # -----------------------------
    # 6. PREP MERGE FRAME
    # -----------------------------
    merge_cols = [case_id_col, disease_col, reasoning_col]

    if context_col:
        df_src[context_out_col] = df_src[context_col].astype(str)
        merge_cols.append(context_out_col)

    # -----------------------------
    # 7. MERGE
    # -----------------------------
    df_final = pd.merge(
        df_res,
        df_src[merge_cols],
        on=case_id_col,
        how="left",
        validate="one_to_one",
    )

    # -----------------------------
    # 8. SAVE
    # -----------------------------
    df_final.to_excel(result_file, index=False)
    print(f"✅ Appended {model_tag} → {result_file}")


In [17]:
from pathlib import Path

DATA_ROOT = Path("/content/drive/MyDrive/Project-AI-Eval/Test-Data/")
OUTPUT_FILE = "ragas_evaluation_dataset.xlsx"

## 2.1 Gwen base

In [18]:
append_gwen_results(
    source_file="gwen_base.xlsx",
    result_file=OUTPUT_FILE,
    model_tag="gwen_base"
)

🔧 Processing model: gwen_base
✅ Appended gwen_base → ragas_evaluation_dataset.xlsx


## 2.2 Gwen finetune

In [19]:
append_gwen_results(
    source_file="gwen_finetune.xlsx",
    result_file=OUTPUT_FILE,
    model_tag="gwen_finetune"
)

🔧 Processing model: gwen_finetune
✅ Appended gwen_finetune → ragas_evaluation_dataset.xlsx


## 2.3 Gwen RAG

In [21]:
append_gwen_results(
    source_file="gwen_rag.xlsx",
    result_file=OUTPUT_FILE,
    model_tag="gwen_rag",
    context_col="context"
)

🔧 Processing model: gwen_rag
✅ Appended gwen_rag → ragas_evaluation_dataset.xlsx


## 2.4 Gwen finetune + RAG

In [22]:
append_gwen_results(
    source_file="gwen_finetune_rag_3.xlsx",
    result_file=OUTPUT_FILE,
    model_tag="gwen_finetune_rag",
    answer_col="pred_answer",
    context_col="rag_trace"
)

🔧 Processing model: gwen_finetune_rag
✅ Appended gwen_finetune_rag → ragas_evaluation_dataset.xlsx


# EXTRA 1: LOAD UNTIDY ANSWER
load answer where format is not deterministic (e.g. mixing answer and reasoning)

In [ ]:
import re
import pandas as pd
from pathlib import Path

def append_gwen_results(
    source_file,
    result_file,
    model_tag: str,
    answer_col: str = "pred_answer",
    case_id_col: str = "case_id",
    context_col: str | None = None,
):
    # -----------------------------
    # 0. NORMALIZE INPUTS
    # -----------------------------
    source_file = Path(source_file)
    result_file = Path(result_file)

    if not source_file.exists():
        raise FileNotFoundError(f"❌ Source file not found: {source_file}")
    if not result_file.exists():
        raise FileNotFoundError(f"❌ Result file not found: {result_file}")

    if not re.match(r"^[a-zA-Z0-9_]+$", model_tag):
        raise ValueError(f"❌ Invalid model_tag: {model_tag}")

    # Define the single output column for the answer
    answer_out_col = f"{model_tag}_answer"
    context_out_col = f"{model_tag}_context" if context_col else None

    print(f"🔧 Processing model: {model_tag}")

    # -----------------------------
    # 1. LOAD SOURCE
    # -----------------------------
    try:
        df_src = pd.read_excel(source_file)
    except Exception:
        df_src = pd.read_csv(source_file)

    # Check for required columns
    for col in [case_id_col, answer_col]:
        if col not in df_src.columns:
            raise KeyError(f"❌ Column '{col}' missing in {source_file}")

    if context_col and context_col not in df_src.columns:
        raise KeyError(f"❌ Context column '{context_col}' missing in {source_file}")

    # -----------------------------
    # 2. PREPARE DATA (Simpler Logic)
    # -----------------------------
    # Ensure ID is string for merging
    df_src[case_id_col] = df_src[case_id_col].astype(str)

    # Directly copy the answer column content instead of parsing
    # We strip whitespace to keep it clean
    df_src[answer_out_col] = df_src[answer_col].astype(str).str.strip()

    # Handle context if requested
    if context_col:
        df_src[context_out_col] = df_src[context_col].astype(str)

    # -----------------------------
    # 3. LOAD RESULT FILE
    # -----------------------------
    try:
        df_res = pd.read_excel(result_file)
    except Exception:
        # Fallback if result file is CSV, though usually it's excel per function name
        df_res = pd.read_csv(result_file)

    if case_id_col not in df_res.columns:
        raise KeyError(f"❌ '{case_id_col}' missing in result file")

    df_res[case_id_col] = df_res[case_id_col].astype(str)

    # -----------------------------
    # 4. OVERWRITE EXISTING COLUMNS
    # -----------------------------
    # Drop the specific columns we are about to add if they already exist
    cols_to_drop = [answer_out_col]
    if context_out_col:
        cols_to_drop.append(context_out_col)

    for col in cols_to_drop:
        if col in df_res.columns:
            print(f"⚠️ Overwriting existing column: {col}")
            df_res = df_res.drop(columns=[col])

    # -----------------------------
    # 5. PREP MERGE FRAME
    # -----------------------------
    merge_cols = [case_id_col, answer_out_col]
    if context_out_col:
        merge_cols.append(context_out_col)

    # -----------------------------
    # 6. MERGE
    # -----------------------------
    df_final = pd.merge(
        df_res,
        df_src[merge_cols],
        on=case_id_col,
        how="left",
        validate="one_to_one",
    )

    # -----------------------------
    # 7. SAVE
    # -----------------------------
    df_final.to_excel(result_file, index=False)
    print(f"✅ Appended {model_tag} → {result_file}")

In [ ]:
append_gwen_results(
    source_file='gwen_finetune_rag_4.xlsx',
    result_file='ragas_evaluation_dataset_finetune_rag_4.xlsx',
    model_tag="gwen_finetune_rag_4",          # This prefix will be added to columns (e.g. gwen_rag_answer)
    case_id_col="test_id",         # Matches the ID column in your CSV
    answer_col="pred_answer",      # The column to grab as the Answer
    context_col="rag_trace"        # The column to grab as the Context
)

🔧 Processing model: gwen_ft_rag_4
✅ Appended gwen_ft_rag_4 → ragas_evaluation_dataset.xlsx


# EXTRA 2: LOAD SEPARATED DISEASE / REASONING

In [23]:
import pandas as pd
from pathlib import Path
import re

def append_results(
    source_file,
    result_file,
    model_tag: str,
    source_disease_col: str = "disease",
    source_reasoning_col: str = "reasoning",
    case_id_col: str = "case_id",
    source_context_col: str | None = "context",  # Pass None if no context
):
    """
    Appends disease and reasoning results from a source file to a master result file.
    No regex extraction is performed; columns are read directly.
    """
    # -----------------------------
    # 0. NORMALIZE INPUTS
    # -----------------------------
    source_file = Path(source_file)
    result_file = Path(result_file)

    if not source_file.exists():
        raise FileNotFoundError(f"❌ Source file not found: {source_file}")
    if not result_file.exists():
        raise FileNotFoundError(f"❌ Result file not found: {result_file}")

    if not re.match(r"^[a-zA-Z0-9_]+$", model_tag):
        raise ValueError(f"❌ Invalid model_tag: {model_tag}")

    # Define output column names for the result file
    out_disease_col = f"{model_tag}_answer_disease"
    out_reasoning_col = f"{model_tag}_answer_reasoning"
    out_context_col = f"{model_tag}_context" if source_context_col else None

    print(f"🔧 Processing model: {model_tag}")

    # -----------------------------
    # 1. LOAD SOURCE
    # -----------------------------
    # Detect format based on extension
    if source_file.suffix.lower() == '.csv':
        df_src = pd.read_csv(source_file)
    else:
        df_src = pd.read_excel(source_file)

    # Check for required columns
    required = [case_id_col, source_disease_col, source_reasoning_col]
    if source_context_col:
        required.append(source_context_col)

    for col in required:
        if col not in df_src.columns:
            raise KeyError(f"❌ Column '{col}' missing in {source_file.name}")

    # -----------------------------
    # 2. PREPARE DATA FOR MERGE
    # -----------------------------
    # Select and rename columns to the final output format
    rename_map = {
        source_disease_col: out_disease_col,
        source_reasoning_col: out_reasoning_col
    }
    if source_context_col:
        rename_map[source_context_col] = out_context_col

    df_to_merge = df_src[[case_id_col] + list(rename_map.keys())].copy()
    df_to_merge = df_to_merge.rename(columns=rename_map)

    # Ensure ID types match (cast to string for safe merging)
    df_to_merge[case_id_col] = df_to_merge[case_id_col].astype(str)

    # -----------------------------
    # 3. LOAD & CLEAN RESULT FILE
    # -----------------------------
    if result_file.suffix.lower() == '.csv':
        df_res = pd.read_csv(result_file)
    else:
        df_res = pd.read_excel(result_file)

    if case_id_col not in df_res.columns:
        raise KeyError(f"❌ '{case_id_col}' missing in result file")

    df_res[case_id_col] = df_res[case_id_col].astype(str)

    # Drop existing columns if we are overwriting them
    cols_to_drop = [out_disease_col, out_reasoning_col]
    if out_context_col:
        cols_to_drop.append(out_context_col)

    for col in cols_to_drop:
        if col in df_res.columns:
            print(f"⚠️ Overwriting existing column: {col}")
            df_res = df_res.drop(columns=[col])

    # -----------------------------
    # 4. MERGE & SAVE
    # -----------------------------
    df_final = pd.merge(
        df_res,
        df_to_merge,
        on=case_id_col,
        how="left",
        validate="one_to_one"
    )

    if result_file.suffix.lower() == '.csv':
        df_final.to_csv(result_file, index=False)
    else:
        df_final.to_excel(result_file, index=False)

    print(f"✅ Successfully appended {model_tag} to {result_file.name}")

In [24]:
append_results(
    source_file="gemini_answer.xlsx",
    result_file="ragas_evaluation_dataset.xlsx",
    model_tag="gemini",
    source_disease_col="disease",
    source_reasoning_col="reasoning",
    source_context_col="context"
)

🔧 Processing model: gemini
✅ Successfully appended gemini to ragas_evaluation_dataset.xlsx
